In [1]:
###IMPORTS

from __future__ import division

import collections
import array
import math

import rhinoinside
rhinoinside.load()
import Rhino

try:
    import Rhino.UI as rui
except ImportError as e:
    raise ImportError("Failed to import Rhino.\n{}".format(e))

IMPORT_EPW

In [2]:
import ladybug.epw as epw

_epw_file_path = "FRA_AC_Agen-La.Garenne.AP.075240_TMYx.epw"

epw_data = epw.EPW(_epw_file_path)
location = epw_data.location #######    
# dry_bulb_temperature = epw_data.dry_bulb_temperature
# dew_point_temperature = epw_data.dew_point_temperature
# relative_humidity = epw_data.relative_humidity
# wind_speed = epw_data.wind_speed
# wind_direction = epw_data.wind_direction
direct_normal_rad = epw_data.direct_normal_radiation #########
diffuse_horizontal_rad = epw_data.diffuse_horizontal_radiation #########
# global_horizontal_rad = epw_data.global_horizontal_radiation
# horizontal_infrared_rad = epw_data.horizontal_infrared_radiation_intensity
# direct_normal_ill = epw_data.direct_normal_illuminance
# diffuse_horizontal_ill = epw_data.diffuse_horizontal_illuminance
# global_horizontal_ill = epw_data.global_horizontal_illuminance
# total_sky_cover = epw_data.total_sky_cover
# barometric_pressure = epw_data.atmospheric_station_pressure
# model_year = epw_data.years
# g_temp = epw_data.monthly_ground_temperature
# ground_temperature = [g_temp[key] for key in sorted(g_temp.keys())]


SKYMATRIX

In [3]:
###IMPORTS SKY MATRIX
import math

try:
    from ladybug_geometry.geometry2d.pointvector import Vector2D
except ImportError as e:
    raise ImportError('\nFailed to import ladybug_geometry:\n\t{}'.format(e))

try:
    from ladybug_radiance.skymatrix import SkyMatrix
except ImportError as e:
    raise ImportError('\nFailed to import ladybug:\n\t{}'.format(e))

try:
    from ladybug_rhino.togeometry import to_vector2d
except ImportError as e:
    raise ImportError('\nFailed to import ladybug_rhino:\n\t{}'.format(e))

try:
    from lbt_recipes.version import check_radiance_date
except ImportError as e:
    raise ImportError('\nFailed to import honeybee_radiance:\n\t{}'.format(e))

# check the istalled Radiance date and get the path to the gemdaymtx executable
check_radiance_date()


In [4]:
###SKY MATRIX
# process and set defaults for all of the global inputs
north_ = 0
_location = location
_direct_rad = direct_normal_rad
_diffuse_rad = diffuse_horizontal_rad
_folder_ = "C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV/cielMatrices"
 
if north_ is not None:  # process the north_
    try:
        north_ = math.degrees(
            to_vector2d(north_).angle_clockwise(Vector2D(0, 1)))
    except AttributeError:  # north angle instead of vector
        north_ = float(north_)

# create the sky matrix object
sky_mtx = SkyMatrix.from_components(
    _location, _direct_rad, _diffuse_rad, 
    #_hoys_, north_, high_density_, ground_r
    )
if _folder_:
    sky_mtx.folder = _folder_

print(sky_mtx.north)

0


#####
ORIENTATION

In [5]:
ORIENTATION_PARCELLE = 0 # [-180 ; 180] ; degrés ; orientation parcelle par rapport à l'axe Nord Sud
HAUTEUR = 3 # [0 ; 100] ; mètres ; hauter de l'axe de rotation, ou du point du PVP le plus bas.
ENTRAXE = 2.5 # [0 ; 10] ; mètres ; distance entre deux rang de panneaux, prises aux axes de rotation dans le cadre du tracking 
RAMPANT = 1.134 # [0 ; 10] mètres ; largeur des panneaux (dimension selon l'axe EST-OUEST)

# ptOri = Rhino.Geometry.Point3d(0,0,0)
# pt1 = Rhino.Geometry.Point3d(1,0,0)
# pt2 = Rhino.Geometry.Point3d(0,1,0)

# plan1 = Rhino.Geometry.Plane(ptOri,pt1,pt2)

ORIENTATION_PARCELLE_RAD = Rhino.RhinoMath.ToRadians(ORIENTATION_PARCELLE)
vecY = Rhino.Geometry.Vector3d.YAxis
vecY.Rotate(angleRadians = ORIENTATION_PARCELLE_RAD, rotationAxis = Rhino.Geometry.Vector3d.ZAxis)
print(vecY)

0,1,0


GRILLE DE POINTS

In [6]:
import Rhino.Geometry as rg
NB_RANGS = 100/ENTRAXE
NB_PVP_RANG = 20
grid_size = 100

nuage = Rhino.Geometry.PointCloud()
column_spacing = grid_size / (NB_RANGS - 1)
row_spacing = grid_size / (NB_PVP_RANG - 1)

for i in range(int(NB_RANGS)):
    for j in range(NB_PVP_RANG):
        x = i * column_spacing
        y = j * row_spacing
        point = Rhino.Geometry.Point3d(x-50, y-50, 1)
        nuage.Add(point)

# for i in nuage.AsReadOnlyListOfPoints():
#     print(i) 

#800 points in this config

####ROTATE POINTCLOUD
nuage.Rotate(ORIENTATION_PARCELLE_RAD, Rhino.Geometry.Vector3d.ZAxis, Rhino.Geometry.Point3d.Origin)

# for i in nuage.AsReadOnlyListOfPoints():
#     print(i)

plans = []
for i in nuage.AsReadOnlyListOfPoints():
    plans.append(Rhino.Geometry)


In [29]:
# Créer un rectangle
plane = rg.Plane.WorldXY
rectangle = rg.Rectangle3d(plane, 10, 5)

# Obtenir les coins du rectangle
corners = [rectangle.Corner(i) for i in range(4)]

# Créer une liste de points pour la polyline
points = Rhino.Collections.Point3dList(5)
for i in range(4):
    points.Add(rg.Point3d(corners[i]))
    print(corners[i])

# Ajouter le premier coin à la fin pour fermer la boucle
points.Add(rg.Point3d(corners[0]))

# Créer une polyline à partir de la liste de points
polyline = rg.Polyline(points)

# Créer une PolylineCurve à partir de la Polyline
curve = rg.PolylineCurve(polyline)

# Créer une extrusion à partir de la courbe
direction = rg.Vector3d(0, 0, 1)  # Dans cet exemple, l'extrusion se fera dans la direction positive de l'axe Z

# Créer une extrusion à partir de la courbe et du vecteur de direction
extrusion = rg.Surface.CreateExtrusion(curve, direction)

print(extrusion.Points)

BB = rg.Brep()
BB2 = rg.Brep()
BB.Append(BB2)


# plane = rg.Plane.WorldXY
# plane.Translate(rg.Vector3d(2, 2, 0))
# rect = rg.Rectangle3d(plane, 10, 5)
        
# # Rotate the rectangle
# rotation = rg.Transform.Rotation(math.pi/2, rg.Vector3d(0, 0, 1), rg.Point3d(2, 2, 0))
# rect.Transform(rotation)
# surface = rg.Surface.CreateFromRectangle(rect)
# brep = rg.Brep.CreateFromSurface(surface)


0,0,0
10,0,0
10,5,0
0,5,0


TypeError: 'NurbsSurfacePointList' object is not subscriptable

In [36]:
def create_rectangles(num_rectangles, x_spacing, y_spacing, width, height, rotation_angle):
    rectangles = []
    for i in range(num_rectangles):
        # Calculate the center of the rectangle
        x = i * x_spacing
        y = i * y_spacing
        
        # Create the rectangle
        plane = rg.Plane.WorldXY
        plane.Translate(rg.Vector3d(x, y, 0))
        rect = rg.Rectangle3d(plane, width, height)
        
        # Rotate the rectangle
        rotation = rg.Transform.Rotation(rotation_angle * i, rg.Vector3d(0, 0, 1), rg.Point3d(x, y, 0))
        rect.Transform(rotation)
        
        rectangles.append(rect)
    
    return rectangles

def convert_to_surfaces(rectangles):
    surfaces = []
    for rect in rectangles:
        surface = rg.Surface.CreateFromRectangle(rect)
        surfaces.append(surface)
    
    return surfaces

def convert_to_breps(surfaces):
    breps = []
    for surface in surfaces:
        brep = rg.Brep.CreateFromSurface(surface)
        breps.append(brep)
    
    return breps

# Parameters
num_rectangles = 5
x_spacing = 20
y_spacing = 15
width = 10
height = 5
rotation_angle = rg.Geometry.PI/4  # 45 degrees in radians

# Create rectangles
rectangles = create_rectangles(num_rectangles, x_spacing, y_spacing, width, height, rotation_angle)

# Convert rectangles to surfaces
surfaces = convert_to_surfaces(rectangles)

# Convert surfaces to breps
breps = convert_to_breps(surfaces)

# Do something with the resulting breps, e.g., add them to the document
for brep in breps:
    rs.AddBrep(brep)


DocumentCollectedException: Rhino.Geometry.Surface
   à Rhino.Runtime.CommonObject.ConstPointer()
   à Rhino.Geometry.GeometryBase.get_HasBrepForm()